#Chapter 10 Ethics and Governance in AI
Chapter 10 introduces the foundations of ethics in AI and the principles of AI governance. We look at the roles of global standards and governing bodies, outline prac-tical ways to govern AI systems across their lifecycle, and review techniques for identifying and mitigating bias. The chapter concludes with an interview with Dr. Francesca Rossi, IBM Fellow and Global Leader for AI Ethics, who connects these ideas to real practice in industry and policy.

#Listing 10-1 Using Great Expectations for Data Quality Checks
This minimal example, which can be expanded into full pipelines, shows how data validation can become an automated, repeatable governance practice.

In [ ]:
!pip install great_expectations pandas scikit-learn

import pandas as pd
import great_expectations as gx
from great_expectations.expectations import (
    ExpectColumnValuesToNotBeNull,
    ExpectColumnValuesToBeBetween,
    ExpectTableRowCountToBeBetween,
)
from sklearn.datasets import load_breast_cancer

# Load a tabular dataset into a DataFrame
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)

# Create a GX Data Context (in-memory)
context = gx.get_context()

# 1. Add a pandas Data Source
data_source = context.data_sources.add_pandas("breast_cancer_source")

# 2. Add a Data Asset for this DataFrame
data_asset = data_source.add_dataframe_asset(name="breast_cancer_asset")

# 3. Add a Batch Definition that uses the whole DataFrame
batch_def = data_asset.add_batch_definition_whole_dataframe("whole_dataframe")

# 4. Pass the in-memory DataFrame as batch parameters
batch = batch_def.get_batch(batch_parameters={"dataframe": df})

# Define expectations
exp1 = ExpectColumnValuesToNotBeNull(column="mean radius")
exp2 = ExpectColumnValuesToBeBetween(
    column="mean texture",
    min_value=0,
    max_value=100,
)
exp3 = ExpectTableRowCountToBeBetween(
    min_value=100,
    max_value=10000,
)

# Validate expectations against the Batch
res1 = batch.validate(exp1)
res2 = batch.validate(exp2)
res3 = batch.validate(exp3)

print("Null check on 'mean radius':", res1.success)
print("Range check on 'mean texture':", res2.success)
print("Row count check:", res3.success)


#Listing 10-2 Opacus Differential Privacy Example
This example illustrates how differential privacy can be incorporated directly into model training workflows.

In [ ]:
!pip install opacus

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

from opacus import PrivacyEngine

# Load a small tabular dataset
data = load_breast_cancer()
X = torch.tensor(data.data, dtype=torch.float32)
y = torch.tensor(data.target, dtype=torch.long)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

train_ds = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)

# Simple classifier
model = nn.Sequential(
    nn.Linear(X_train.shape[1], 32),
    nn.ReLU(),
    nn.Linear(32, 2)
)

optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
criterion = nn.CrossEntropyLoss()

# Attach differential privacy using Opacus
privacy_engine = PrivacyEngine()

model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    epochs=3,
    target_epsilon=5.0,
    target_delta=1e-5,
    max_grad_norm=1.0,
)

# Training loop (very small, for illustration)
model.train()
for epoch in range(3):
    for xb, yb in train_loader:
        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()

epsilon = privacy_engine.get_epsilon(delta=1e-5)
print(f"Training finished with ε = {epsilon:.2f}, δ = 1e-5")


#Listing 10-3 Measuring Group-wise Fairness with Fairlearn
This program trains a simple classifier, then uses Fairlearn to compute accuracy and selection rate for each group. It shows how a model that looks acceptable overall can still behave differently across groups, which is central to fairness analysis and governance.

In [ ]:
!pip install fairlearn scikit-learn

import numpy as np
import pandas as pd

from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score          # <-- from sklearn.metrics
from fairlearn.metrics import MetricFrame, selection_rate  # <-- only these from fairlearn

# Synthetic dataset with a binary label
X, y = make_classification(
    n_samples=2000,
    n_features=10,
    n_informative=5,
    random_state=42
)

# Synthetic "group" attribute (for example, two demographic groups)
rng = np.random.default_rng(42)
group_attr = rng.integers(0, 2, size=y.shape[0])  # values 0 or 1

X_train, X_test, y_train, y_test, g_train, g_test = train_test_split(
    X, y, group_attr, test_size=0.3, random_state=42, stratify=y
)

# Train a simple classifier
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# Predictions
y_pred = clf.predict(X_test)

# Use Fairlearn to compute metrics by group
metric_frame = MetricFrame(
    metrics={
        "accuracy": accuracy_score,
        "selection_rate": selection_rate,
    },
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=g_test,
)

print("Overall accuracy:", accuracy_score(y_test, y_pred))
print("\nAccuracy by group:")
print(metric_frame.by_group["accuracy"])

print("\nSelection rate by group:")
print(metric_frame.by_group["selection_rate"])


#Listing 10-4 Using SHAP to Identify Influential Features
This listing trains a random forest on a tabular dataset and uses SHAP to rank the most influential features. By computing the mean absolute SHAP value for each fea-ture, the example highlights which inputs contribute most strongly to the model’s outputs. This approach provides a practical, interpretable summary of model behavior and helps identify cases where a model might be relying on inappropriate signals or proxies for sensitive attributes. In real deployments, domain-specific feature names often make these explanations even more informative.

In [ ]:
!pip install shap scikit-learn pandas matplotlib

import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Load a tabular dataset with multiple informative features
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Train a tree-based classifier
model = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    random_state=42,
)
model.fit(X_train, y_train)

# Create SHAP explainer and compute values on the test set
explainer = shap.TreeExplainer(model)
X_sample = X_test

shap_values = explainer.shap_values(X_sample)

# For a binary classifier, shap_values is a list (one array per class)
if isinstance(shap_values, list):
    shap_values_class_1 = shap_values[1]   # SHAP values for positive class
else:
    shap_values_class_1 = shap_values

# Ensure we have (n_samples, n_features) by collapsing any extra dims
shap_values_class_1 = np.array(shap_values_class_1)
if shap_values_class_1.ndim > 2:
    # e.g., (n_samples, n_features, 1) -> average over last axis
    shap_values_class_1 = shap_values_class_1.mean(axis=-1)

# Compute mean absolute SHAP value per feature -> shape (n_features,)
mean_abs_shap = np.mean(np.abs(shap_values_class_1), axis=0)
mean_abs_shap = np.asarray(mean_abs_shap, dtype=float).ravel()

feature_names = np.array(X_sample.columns)

# Sort features by importance (descending) and select top 10
order = np.argsort(mean_abs_shap)[::-1]
top_k = min(10, len(mean_abs_shap))
top_idx = order[:top_k]

top_features = feature_names[top_idx]
top_importances = mean_abs_shap[top_idx]   # 1-D float array of length top_k

print("Top features by mean absolute SHAP value:")
for name, val in zip(top_features, top_importances):
    print(f"{name}: {val:.4f}")

# Plot as a horizontal bar chart
plt.figure(figsize=(8, 5))
positions = np.arange(len(top_features))
plt.barh(positions, top_importances)
plt.yticks(positions, top_features)
plt.gca().invert_yaxis()  # highest importance at the top
plt.xlabel("mean(|SHAP value|) (average impact on model output)")
plt.title("Top features by mean absolute SHAP value")
plt.tight_layout()
plt.show()


#Listing 10-5 Tracking Model Training and Metrics with MLflow
This example shows how to log a model training run with MLflow. The code demonstrates how systematic experiment tracking can strengthen governance by making it easier to reproduce results, audit model behavior, and understand how a model was built.

In [ ]:
!pip install mlflow

import mlflow
import mlflow.sklearn

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

# Load a small dataset
data = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target,
    test_size=0.3,
    random_state=42,
    stratify=data.target
)

# Start an MLflow run
with mlflow.start_run(run_name="rf_governance_example"):
    n_estimators = 100
    max_depth = 5

    # Log parameters
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)

    # Train model
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )
    model.fit(X_train, y_train)

    # Evaluate and log metrics
    y_prob = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_prob)
    mlflow.log_metric("roc_auc", auc)

    # Log the model artifact with an input example
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="model",
        input_example=X_train[:5]
    )

    print("Logged run with ROC AUC:", auc)
